# Upravljanje memorijom

Cilj ove sveske je da ukaže na neke zanimljive koncepte biblioteke `numpy` koji se tiču upravljanja memorijom. 

In [1]:
import numpy as np

![ndarray struktura](assets/memory_management.png)

Na slici je prikazan jedan višedimenzioni niz oblika 3x3 koji sadrži brojeve od 0 do 8. Njega možemo kreirati pozivom funkcije `array`. Videli smo da broj njegovih dimenzija i sam oblik možemo pročitati svojstvima `ndim` i `shape`, dok sam tip podataka sadržanih u nizu i njihove pojedinačne memorijske zahteve možemo pročitati pomoću svojstava  `dtype` i `itemsize`. 

In [2]:
M = np.array([
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
])

In [3]:
M

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [4]:
print('Broj dimenzija: ', M.ndim)

Broj dimenzija:  2


In [5]:
print('Oblik: ', M.shape)

Oblik:  (3, 3)


In [6]:
print('Tip podataka: ', M.dtype)

Tip podataka:  int64


In [7]:
print('Memorijski zahtevi pojedinacnih elemenata (u bajtovima): ', M.itemsize)

Memorijski zahtevi pojedinacnih elemenata (u bajtovima):  8


Ovom višedimenzionom nizu odgovara i jedan blok memorije u kojem su elementi smešteni uzastopno jedan do drugog. Informacije o memorijskom rasporedu elemenata, kao i o svojstvima fizičkog bloka memorije koji se koristi za čuvanje elemenata mogu se dobiti korišćenjem svojstva `flags`.

In [8]:
M.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

Svojstvo `C_CONTIGUOUS` označava da se vrste matrice ređaju u memoriji jedna za drugom (C u imenu svojstva naglasava da je ovo ponašanje nalik ponašanju u jeziku C). Sa druge strane, svojstvo `F_CONTIGUOUS` ukazuje na to da su kolone matrice postavljene u memoriji jedna za drugom (F u imenu svojstva ukazuje na sličnost sa programskim jezikom Fortran). Raspored elemenata je važan za izvođenje operacija na niskom nivou.

Ukoliko svojstvo `OWNDATA` ima vrednost `True`, višedimenzioni niz ima svoj fizički blok u memoriji. Ukoliko je vrednost ovog svojstva `False`, višedimenzioni niz predstavlja samo **pogled** na neki postojeći blok. Ovakva organizacija memorije je, takođe, važna zbog brzog i efikasnog izvođenja operacija. 

Na primer, transponovani višedimenzioni niz polaznog niza `M` ili niz dobijen primenom operatora isecanja su primeri nizova pogleda.

In [9]:
M_transposed = M.transpose()

In [10]:
M_transposed.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [11]:
M_transposed

array([[0, 3, 6],
       [1, 4, 7],
       [2, 5, 8]])

In [12]:
M_reduced = M[1:3, :]

In [13]:
M_reduced.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [14]:
M_reduced

array([[3, 4, 5],
       [6, 7, 8]])

Zanimljivo je prokomentarisati i promenu vrednosti `C_CONTIGUOUS` i `F_CONTIGUOUS` zastavica u slučaju transponovanja: zarad efikasnijeg izvršavanja, ne dolazi do fizičke reorganizacije elemenata već samo do promene prateće strukture pridružene nizu tako da se sada elementi iz pridruženog memorijskog bloka očitavaju po kolonama, a ne po vrstama.  

Prilikom rada sa bibliotekom `numpy` treba voditi računa o tome da li funkcije koje se koriste kreiraju nove memorijske blokove ili koriste poglede nad postojećim. Na primer, funkcije `flatten` i `ravel` se mogu koristiti za svođenje višedimenzionog niza na vektor vrednosti. 

In [15]:
M_flatten = M.flatten()

In [16]:
M_flatten

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [17]:
M_flatten.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [18]:
M_ravel = M.ravel()

In [19]:
M_ravel

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [20]:
M_ravel.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

Kao što možemo videti efekat primene ovih dveju funkcija je isti uz bitnu razliku da funkcija `flatten` pravi kopiju postojećeg niza, a funkcija `ravel` pogled na njega.

Funkcijom `shares_memory` se može proveriti da li dva višedimenziona niza dele memoriju ili ne.

In [21]:
np.shares_memory(M, M_flatten)

False

In [22]:
np.shares_memory(M, M_ravel)

True

Eksplicitno kopiranje memorijskog bloka se može postići koristeći funkciju `copy`.

In [23]:
M_copy = np.copy(M)

In [24]:
M_copy

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [25]:
M_copy.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

Struktura koja je pridružena višedimenzionom nizu ima još jedno zanimljivo svojstvo koje se zove `strides`. Svojstvo `strides` oslikava broj bajtova potrebnih da se pređe sa jednog elementa na drugi duž svake od dimenzija višedimenzionog niza. Tako, na primer, da bi se u višedimenzionom nizu `M` pročitao element sledeće vrste potreban je pomeraj od 24 bajta (24=3x8), a da bi se pročitao element sledeće kolone pomeraj od 8 bajtova (8=1x8) jer je dimenzija niza 3x3, veličina jednog elementa 8 bajtova.

In [26]:
M.strides

(24, 8)

Tako se, recimo, prilikom transponovanja višedimenzionog niza uz promene pomenutih  `C_CONTIGUOUS` i `F_CONTIGUOUS` zastavica, razmenjuju vrednosti `strides` svojstva.

In [27]:
M_transposed.strides

(8, 24)

Biblioteka `numpy` raspolaže i posebnim APIjem u okviru kojeg se može koristiti funkcija `as_strided` koja za različita podešavanja oblika i pomeraja kreira poglede. Oblik željenog niza se zadaje argumentom `shape` (on će predstavljati `shape` svojstvo novog niza) dok se pomeraj zadaje argumentom `strides` koji predstavlja `strides` svojstvo novog niza u odnosu na zadati memorijski blok.

Tako, recimo, očitavanje elemenata prve vrste možemo postići na sledeći način:

In [28]:
itemsize = M.itemsize

In [29]:
M[0, :]

array([0, 1, 2])

In [30]:
np.lib.stride_tricks.as_strided(M, shape=(3, ), strides=(1*itemsize,))

array([0, 1, 2])

Očitavanje svakog drugog elementa prve vrste korišćenjem operatora isecanja možemo da realizujemo sledećom naredbom:

In [31]:
M[0, ::2]

array([0, 2])

Isti rezultat možemo dobiti korišćenjem funkcije `as_strided` sa sledećim argumentima: 

In [32]:
np.lib.stride_tricks.as_strided(M, shape=(2, ), strides=(2*itemsize,))

array([0, 2])

Očitavanje dijagonalnih elemenata je moguće ostvariti na sledeći način:

In [33]:
np.lib.stride_tricks.as_strided(M, shape=(3,), strides=(32,))

array([0, 4, 8])

Još neke zanimljive primere rada sa funkcijom `as_strided` možete pronaći u [ovom](https://towardsdatascience.com/advanced-numpy-master-stride-tricks-with-25-illustrated-exercises-923a9393ab20) članku.

Podsetimo i priče o osama višedimenzionih nizova koje se koriste u nekim funkcija, poput `sum`, `average` ili `max`, za redukciju vrednosti duž određene dimenzije. Interpretacija osa zavisi od broj dimenzija. Na slici možemo videti značenja za nizove dimenzije jedan, dva i tri. Blokovima dimenzije jedan možemo predstavljati, na primer, merenja u vremenu, blokovima dimenzije dva tabelarne podatke ili crno-bele slike, dok blokovima dimenzija tri se, na primer, mogu predstavljati slike u boji ili tekstualni podaci (naučićemo kako). Blokovima dimenzije četiri se mogu prikazivati video zapis i slično. 

![ndarray axis](assets/numpy_axis.png)

Tako se, recimo, zbir elemenata matrice M po kolonama može dobiti na sledeći način: 

In [34]:
np.sum(M, axis=0)

array([ 9, 12, 15])

Zbir elemenata matrice M po vrstama se može dobiti na sledeći način:

In [35]:
np.sum(M, axis=1)

array([ 3, 12, 21])

U priči o efikasnom izvođenju operacija, uvek se pominje i `broadcasting`, ponašanje Python interpretera kojim pokušava sa svođenjem na iste dimenzije dva ili više višedimenzionih nizova. Ovo se posebno odnosi na nizove koji imaju različite oblike (svojstvo `shape`) pa izvođenje nekih operacija umesto očekivane greške može rezultirati neočekivanim rezultatom.

Tako je, na primer, moguće: 
* sabrati višedimenzioni niz i skalar

In [36]:
np.arange(3) + 5

array([5, 6, 7])

* sabirati višedimenzioni niz i vektor sa kompatibilnim brojem elemenata

In [37]:
np.ones((3, 3)) + np.arange(3)

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

* sabrati višedimenzioni niz sa jednom kolonom i vektor sa kompatibilnim brojem elemenata

In [38]:
np.arange(3).reshape((3, 1)) + np.arange(3)

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

Vizualizacije koje odgovaraju navedenim primerima su prikazane niže. Bledo siva proširenja se odnose na emitovanje dimenzija prilikom izvođenja operacija. Važno je naglasiti da su ova proširivanja virtuelna tj. da se ne menjaju fizičke dimenzije memorijskih blokova u kojima se vrednosti nalaze.

![ndarray broadcasting](assets/numpy_broadcasting.png)

Više o ovom svojstvu biblioteke `numpy` se može pročitati u [zvaničnoj dokumentaciji](https://numpy.org/doc/stable/user/basics.broadcasting.html).